# USDA - National Agricultural Statistics Service

### Import Python tools and Jupyter configuration

In [1]:
%load_ext lab_black

In [2]:
import pandas as pd
import geopandas as gpd
import altair as alt
from datetime import timedelta
import numpy as np

In [3]:
pd.options.display.max_columns = 100
pd.options.display.max_rows = 1000
pd.options.display.max_colwidth = None

In [4]:
src = pd.read_csv(
    "data/raw/lettuce_acres_harvested_county_2017.csv",
    dtype={"County ANSI": "str", "State ANSI": "str"},
)

In [5]:
src.columns = src.columns.str.lower().str.replace(" ", "_", regex=True)

In [6]:
src_slim = src[src["data_item"] == "LETTUCE - ACRES HARVESTED"][
    [
        "year",
        "geo_level",
        "state",
        "state_ansi",
        "county",
        "county_ansi",
        "data_item",
        "value",
    ]
]

In [7]:
src_slim.rename(
    columns={"state_ansi": "state_fips", "county_ansi": "county_fips"}, inplace=True
)

In [8]:
src_slim["geo_level"] = src_slim["geo_level"].str.title()
src_slim["county"] = src_slim["county"].str.title()
src_slim["state"] = src_slim["state"].str.title()
src_slim["data_item"] = src_slim["data_item"].str.title()

In [9]:
src_slim.head()

,year,geo_level,state,state_fips,county,county_fips,data_item,value
0,2017,County,Alabama,01,Elmore,051,Lettuce - Acres Harvested,8
8,2017,County,Alabama,01,Hale,065,Lettuce - Acres Harvested,(D)
24,2017,County,Alabama,01,Macon,087,Lettuce - Acres Harvested,(D)
32,2017,County,Alabama,01,Baldwin,003,Lettuce - Acres Harvested,(D)
44,2017,County,Alabama,01,Escambia,053,Lettuce - Acres Harvested,(D)


### Strip out withheld values

In [10]:
# D = Withheld to avoid disclosing data for individual operations.
# Z = Less than half the rounding unit.

In [11]:
src_slim["value"] = (
    src_slim["value"]
    .str.strip(" ")
    .str.replace("(D)", "0", regex=False)
    .str.replace("(Z)", "0", regex=False)
    .str.replace(",", "", regex=False)
)

In [12]:
src_slim["value"] = src_slim["value"].astype(int)

In [13]:
src_slim["fips"] = src_slim["state_fips"] + src_slim["county_fips"]

In [14]:
df = src_slim.copy()

---

In [15]:
df.head()

,year,geo_level,state,state_fips,county,county_fips,data_item,value,fips
0,2017,County,Alabama,01,Elmore,051,Lettuce - Acres Harvested,8,01051
8,2017,County,Alabama,01,Hale,065,Lettuce - Acres Harvested,0,01065
24,2017,County,Alabama,01,Macon,087,Lettuce - Acres Harvested,0,01087
32,2017,County,Alabama,01,Baldwin,003,Lettuce - Acres Harvested,0,01003
44,2017,County,Alabama,01,Escambia,053,Lettuce - Acres Harvested,0,01053


In [16]:
df.tail()

,year,geo_level,state,state_fips,county,county_fips,data_item,value,fips
20048,2017,County,Wyoming,56,Washakie,043,Lettuce - Acres Harvested,0,56043
20056,2017,County,Wyoming,56,Albany,001,Lettuce - Acres Harvested,0,56001
20064,2017,County,Wyoming,56,Platte,031,Lettuce - Acres Harvested,0,56031
20072,2017,County,Wyoming,56,Lincoln,023,Lettuce - Acres Harvested,0,56023
20088,2017,County,Wyoming,56,Teton,039,Lettuce - Acres Harvested,1,56039


In [17]:
df[df["state"] == "California"]

,year,geo_level,state,state_fips,county,county_fips,data_item,value,fips
752,2017,County,California,06,Alameda,001,Lettuce - Acres Harvested,2,06001
768,2017,County,California,06,Contra Costa,013,Lettuce - Acres Harvested,0,06013
776,2017,County,California,06,Lake,033,Lettuce - Acres Harvested,0,06033
784,2017,County,California,06,Marin,041,Lettuce - Acres Harvested,100,06041
800,2017,County,California,06,Monterey,053,Lettuce - Acres Harvested,169496,06053
816,2017,County,California,06,Napa,055,Lettuce - Acres Harvested,4,06055
832,2017,County,California,06,San Benito,069,Lettuce - Acres Harvested,2688,06069
848,2017,County,California,06,San Luis Obispo,079,Lettuce - Acres Harvested,2907,06079
864,2017,County,California,06,San Mateo,081,Lettuce - Acres Harvested,58,06081
880,2017,County,California,06,Santa Clara,085,Lettuce - Acres Harvested,0,06085


In [18]:
df.sort_values("value", ascending=False).head(20)

,year,geo_level,state,state_fips,county,county_fips,data_item,value,fips
800,2017,County,California,06,Monterey,053,Lettuce - Acres Harvested,169496,06053
1352,2017,County,California,06,Imperial,025,Lettuce - Acres Harvested,35997,06025
1448,2017,County,California,06,Santa Barbara,083,Lettuce - Acres Harvested,15745,06083
1096,2017,County,California,06,Fresno,019,Lettuce - Acres Harvested,4702,06019
896,2017,County,California,06,Santa Cruz,087,Lettuce - Acres Harvested,4450,06087
1400,2017,County,California,06,Riverside,065,Lettuce - Acres Harvested,3189,06065
848,2017,County,California,06,San Luis Obispo,079,Lettuce - Acres Harvested,2907,06079
832,2017,County,California,06,San Benito,069,Lettuce - Acres Harvested,2688,06069
1464,2017,County,California,06,Ventura,111,Lettuce - Acres Harvested,2257,06111
1688,2017,County,Colorado,08,Saguache,109,Lettuce - Acres Harvested,1746,08109


---

### Export

In [19]:
df.to_csv("data/processed/lettuce_acres_harvested_counties_2017.csv", index=False)